<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#一、外资周度净流入" data-toc-modified-id="一、外资周度净流入-1">一、外资周度净流入</a></span></li><li><span><a href="#二、融资余额周度变动" data-toc-modified-id="二、融资余额周度变动-2">二、融资余额周度变动</a></span></li><li><span><a href="#三、外资与融资余额流入关系" data-toc-modified-id="三、外资与融资余额流入关系-3">三、外资与融资余额流入关系</a></span></li></ul></div>

In [1]:
from it_data_tool.data_tool import *
from lowrisk_it_tools.WindPy import *
from macrotoolchain.mainAPI import *
di.display_html(getCatalogHtml(),raw=True)
from lowrisk.CustomPlot import *
from datetime import datetime,timedelta
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
h1Title('资金流入流出')
h2Title('更新时间： {}年{}月{}日'.format(Date.today().year, Date.today().month, Date.today().day))

资金流入流出

更新时间： 2022年10月26日

# 一、外资周度净流入

In [2]:
# 北上资金
end_date = datetime.today()
start_date = '20200101'
end_date = end_date.strftime('%Y%m%d')

In [3]:
def get_stock_price(sd,ed):
    sql = '''
        select wind_code as scode, record_date as trade_dt, group_type, inoutflow as jingmairu
        from ads_stock_capital_inoutflow_a
        where record_date >= '{}'
        '''.format(sd)
    df = get_quant_db_hz(sql)
    df = df.groupby(['scode','trade_dt','group_type'])['jingmairu'].sum().reset_index()
    df['jingmairu'] /= 1e8
    df['trade_dt'] = pd.to_datetime(df['trade_dt'])
    # 增加行业
    sql = '''
        select wind_code as scode, citic_industry as industry
        from ads_wind_ahshareciticsindustries_style_a
        '''
    ind = get_quant_db_hz(sql)
    df = pd.merge(df,ind,on=['scode'],how='left')
    df['industry'] = df['industry'].fillna('综合')
    # 行业和中信一级行业对齐
    df.loc[df.industry=='保险Ⅱ','industry'] = '非银行金融'
    df.loc[df.industry=='证券Ⅱ','industry'] = '非银行金融'
    df.loc[df.industry=='多元金融','industry'] = '非银行金融'
    df.loc[df.industry=='酒类','industry'] = '食品饮料'
    df.loc[df.industry=='食品','industry'] = '食品饮料'
    df.loc[df.industry=='饮料','industry'] = '食品饮料'
    return df

waizi_specific = get_stock_price(start_date,end_date)

In [4]:
citic_name = ['石油石化','基础化工','煤炭','钢铁','有色金属','建筑','建材','农林牧渔','电力及公用事业','电力设备及新能源',
              '汽车','机械','交通运输','轻工制造','国防军工','商贸零售','食品饮料','纺织服装','家电','消费者服务','医药',
              '电子','计算机','通信','传媒','银行','非银行金融','房地产','综合金融','综合']

In [5]:
def cal_waizi_net(df,name):
    if name!='北上资金总量':
        df = df[df.group_type==name].copy()
    df = df.groupby(['trade_dt','industry'])['jingmairu'].sum().reset_index()
    df = df.pivot_table(index=['trade_dt'],columns=['industry'],values=['jingmairu'])['jingmairu']
    df = df.resample('W').sum()
    df = df.iloc[len(df)-20:,:].T
    df = np.round(df.loc[citic_name,:],2)
    df.columns = [x.strftime('%Y-%m-%d') for x in df.columns]
    new = df.columns[-1]
    df = df.sort_values([new],ascending=False)
    return df.iloc[:,-12:]

In [6]:
beishang_net = cal_waizi_net(waizi_specific,'北上资金总量')
print('北上资金总量周度净流入')
beishang_net.style.bar(align='mid',color=['mediumaquamarine','lightcoral'],axis=0)

北上资金总量周度净流入


,2022-08-14,2022-08-21,2022-08-28,2022-09-04,2022-09-11,2022-09-18,2022-09-25,2022-10-02,2022-10-09,2022-10-16,2022-10-23,2022-10-30
industry,,,,,,,,,,,,
有色金属,6.07,10.36,4.29,-26.25,53.51,-2.02,-7.2,-11.74,0,6.85,-8.92,7.72
煤炭,6.9,5.29,-11.39,6.17,-4.9,6.08,-7.49,2.13,0,-4.73,8.57,7.35
房地产,0.81,5.64,-0.5,10.23,7.48,-4.12,6.3,-7.76,0,-6.96,-0.74,4.61
石油石化,7.94,1.26,-4.27,-0.46,-3.36,-1.53,-3.89,7.06,0,3.66,-5.83,3.46
汽车,17.27,0.69,3.04,10.86,-1.43,-2.15,4.23,-6.13,0,-10.19,-8.13,1.96
消费者服务,-0.94,-2.7,-29.37,-10.28,1.62,-3.66,-8.84,2.28,0,-1.62,-12.49,1.1
医药,-0.66,-18.35,-15.41,1.86,11.85,-8.29,-17.42,38.51,0,12.25,-9.36,0.95
综合,1.13,-0.85,1.13,0,-1.03,-1.67,-0.05,0.99,0,-0.16,-0.99,0.1
纺织服装,-0.48,0.39,0.9,0.08,-0.17,0.62,1.45,0.33,0,1.02,0.35,-0.16


In [7]:
beishang_net1 = cal_waizi_net(waizi_specific,'配置')
print('配置型资金周度净流入')
beishang_net1.style.bar(align='mid',color=['mediumaquamarine','lightcoral'],axis=0)

配置型资金周度净流入


,2022-08-14,2022-08-21,2022-08-28,2022-09-04,2022-09-11,2022-09-18,2022-09-25,2022-10-02,2022-10-09,2022-10-16,2022-10-23,2022-10-30
industry,,,,,,,,,,,,
电力及公用事业,-0.54,-4.57,-0.16,3.69,-4.58,0.02,-1.74,-7.63,0,-5.15,-2.92,11.02
国防军工,0.13,-0.4,-0.49,0.55,0.26,1.12,0.46,-0.14,0,2.15,2.97,1.25
房地产,-2,-1.04,1.51,3.18,3.01,-0.79,-4.08,-1.25,0,-1.34,-3.45,0.27
通信,12.5,0.09,-1.59,-2.3,3.22,-2.15,3.24,4.2,0,0.42,2.22,0.14
纺织服装,0.03,-0.02,0.24,-0.09,-0.24,0.23,-0.33,0.08,0,-0.13,0.22,0.02
商贸零售,0.31,0.19,0.22,-0.15,-0.21,-0.02,0.23,-0.33,0,-0.43,-0.58,0.01
综合金融,0.08,0.16,0.04,-0.06,-0.21,0.26,0.08,-0.15,0,-0.09,-0.02,-0
综合,0.05,0.03,-0.05,-0.05,0.01,-0.28,0.11,0.05,0,0.03,-0.18,0
建筑,-0.72,0.48,3,0.51,-0,0.86,1.61,-2.37,0,-3.15,0.33,-0.07


In [8]:
beishang_net1 = cal_waizi_net(waizi_specific,'投机')
print('交易型资金周度净流入')
beishang_net1.style.bar(align='mid',color=['mediumaquamarine','lightcoral'],axis=0)

交易型资金周度净流入


,2022-08-14,2022-08-21,2022-08-28,2022-09-04,2022-09-11,2022-09-18,2022-09-25,2022-10-02,2022-10-09,2022-10-16,2022-10-23,2022-10-30
industry,,,,,,,,,,,,
医药,7.35,-11.41,-18.1,1.82,14.37,-3.77,-8.75,34.87,0,11.86,-12.34,16.14
有色金属,9.73,5.37,6.3,-32.68,43.51,-7.29,-6.83,-9.08,0,4.75,-8.61,8.3
煤炭,6.53,4.97,-9.34,3.72,-5.29,4.04,-9.6,2.74,0,-0.7,6.24,7.84
食品饮料,-1.89,5.09,19.32,-26.71,-8.46,1.02,-0.87,30.01,0,-41.35,-49.55,4.95
房地产,2.48,6.85,-2.07,7,4.4,-2.53,10.17,-6.66,0,-5.29,2.04,4.33
石油石化,10.1,3.98,-1.4,0.37,0.44,-3.37,-6.98,3.98,0,6.76,-5.17,3.84
汽车,12.56,2.07,4.63,7.81,0.07,-1.09,2.64,-2.44,0,-0.6,-2.31,2.66
消费者服务,2.06,-1.45,-6.35,-1.13,1.36,0.59,-5.46,1.61,0,-0.56,-5.99,2.64
交通运输,2.87,12.83,5.58,-6.06,3.48,-0.74,-10.86,2.95,0,-4.96,-4.44,0.72


In [9]:
beishang_net1 = cal_waizi_net(waizi_specific,'内地')
print('托管于内港资金周度净流入')
beishang_net1.style.bar(align='mid',color=['mediumaquamarine','lightcoral'],axis=0)

托管于内港资金周度净流入


,2022-08-14,2022-08-21,2022-08-28,2022-09-04,2022-09-11,2022-09-18,2022-09-25,2022-10-02,2022-10-09,2022-10-16,2022-10-23,2022-10-30
industry,,,,,,,,,,,,
食品饮料,-1.13,0.17,-0.81,-1.5,-7,-2.58,0.13,1.71,0,-0.79,1,1.11
有色金属,0.92,1.96,-2.66,-1.66,2.74,1.24,-2.68,-1.56,0,-1.31,-0.06,0.74
建材,0.32,0.58,-0.09,-0.8,-0.26,-0.05,0.31,-0.11,0,-0.17,-0.23,0.45
石油石化,0.06,1.28,-0.83,0.62,0.81,-0.23,-1.57,0.25,0,-0.12,-0.08,0.31
电子,2.27,1.72,-1.55,-1.25,-2.31,0.14,-2.89,-2.11,0,-0.5,-0.83,0.24
建筑,-0.42,0.39,0.22,0.27,0.6,-0.7,-0.08,-0.28,0,-0.51,0.06,0.14
电力及公用事业,0.26,-1.17,1.58,-1.89,0.96,-0.71,-0.94,-0.98,0,-3.68,-0.89,0.14
轻工制造,0.26,0.11,-0.24,0.26,-0.07,-0.02,-0.2,-0.13,0,0.21,-0.17,0.07
传媒,0.42,-0.32,-0.02,0.18,-0.27,-0.16,-0.01,0.05,0,-0.43,-0.11,0.03


# 二、融资余额周度变动

In [10]:
start_date = (datetime.today()-timedelta(150)).strftime('%Y%m%d')

def get_rzye(sd):
    sql = '''
        select s_info_windcode as scode, trade_dt, s_margin_tradingbalance as guimo
        from ads_wind_asharemargintrade_a
        where trade_dt>='{}'
        order by s_info_windcode,trade_dt
        '''.format(sd)
    df = get_quant_db_hz(sql)
    df.trade_dt = pd.to_datetime(df.trade_dt)
    df['guimo'] /= 1e8
    # 增加行业
    sql = '''
        select wind_code as scode, citic_industry as industry
        from ads_wind_ahshareciticsindustries_style_a
        '''
    ind = get_quant_db_hz(sql)
    df = pd.merge(df,ind,on=['scode'],how='left')
    df['industry'] = df['industry'].fillna('综合')
    # 行业和中信一级行业对齐
    df.loc[df.industry=='保险Ⅱ','industry'] = '非银行金融'
    df.loc[df.industry=='证券Ⅱ','industry'] = '非银行金融'
    df.loc[df.industry=='多元金融','industry'] = '非银行金融'
    df.loc[df.industry=='酒类','industry'] = '食品饮料'
    df.loc[df.industry=='食品','industry'] = '食品饮料'
    df.loc[df.industry=='饮料','industry'] = '食品饮料'
    return df

rzye = get_rzye(start_date)

In [11]:
def cal_rzye_net(df):
    df1 = df.pivot_table(index=['trade_dt'],columns=['scode'],values=['guimo'])['guimo']
    df1 = df1.diff().resample('W').sum()
    df1 = df1.reset_index().melt(id_vars=['trade_dt']).dropna().rename(columns={'value':'jlr'})
    info = df[['scode','industry']].drop_duplicates()
    df1 = pd.merge(info,df1,on=['scode'])
    df1 = df1.groupby(['trade_dt','industry']).sum().reset_index()
    df1 = df1.pivot_table(index=['industry'],columns=['trade_dt'],values=['jlr'])['jlr']
    need_list = df1.columns[-20:]
    df1 = np.round(df1.loc[citic_name,need_list],2)
    df1.columns = [x.strftime('%Y-%m-%d') for x in df1.columns]
    new = df1.columns[-1]
    df1 = df1.sort_values([new],ascending=False)
    return df1.iloc[:,-12:]

rzye_net = cal_rzye_net(rzye)

In [12]:
rzye_net.style.bar(align='mid',color=['mediumaquamarine','lightcoral'],axis=0)

,2022-08-14,2022-08-21,2022-08-28,2022-09-04,2022-09-11,2022-09-18,2022-09-25,2022-10-02,2022-10-09,2022-10-16,2022-10-23,2022-10-30
industry,,,,,,,,,,,,
综合,-1.09,0.77,-8.52,4.75,-7.43,1.16,-6.32,-19.9,0,0.53,-1.43,7.31
食品饮料,3.67,6.64,-0.17,2.32,-0.15,-5.62,-8.64,-19.53,0,9.28,12.15,5.64
电力设备及新能源,16.23,23.27,-19.4,-39.8,4.99,-9.04,2.61,-38.6,0,5.23,6.55,2.79
建筑,-1.04,-0.41,-3.11,-2.53,-2.87,-1.38,0.3,-6.88,0,-0.57,-1.6,2.39
机械,13.85,7.58,-5.17,-12.28,2.98,-4.41,-3.6,-11.02,0,-3.55,1.83,2.02
银行,-3.14,1.1,-6.26,-2.74,-13.83,-6.49,-3.4,-4.39,0,5.55,-0.68,1.34
消费者服务,-3.23,1.64,0.63,0.54,-3.22,-1.73,-0.69,-2.92,0,-0.87,3.1,0.88
汽车,3.75,3.34,-21.18,-17.76,1.06,-0.47,-13.76,-13.24,0,0.19,0.89,0.59
家电,0.07,1.85,-3.08,-7.16,0.22,-3.86,1.61,-4.09,0,-0.52,3.85,0.48


# 三、外资与融资余额流入关系

In [13]:
# 指数成分股
name = pd.read_excel('行业名称.xlsx')
code_list = list(set(name['代码'].tolist()))

def get_index_chengfen():
    df = pd.DataFrame()
    for c in code_list:
        data = w.wset("sectorconstituent","date=2022-07-22;windcode={}".format(c),usedf=True)[1]
        data['代码'] = c
        data = data.rename(columns={'wind_code':'scode'}).drop(['date','sec_name'],axis=1)
        df = pd.concat([df,data],axis=0)
    df = pd.merge(name,df,on=['代码'])
    return df

index_chengfen = get_index_chengfen()

In [14]:
def get_hebing(df1,df2,name,days):
    # 维修北上资金
    df1 = df1.copy()
    df1 = pd.merge(df1,index_chengfen,on=['scode']).drop(['industry','指数分类','代码'],axis=1).rename(columns={'指数简称':'industry'})
    if name!='北上资金':
        df1 = df1[df1.group_type==name].copy()
    df1 = df1.groupby(['trade_dt','industry'])['jingmairu'].sum().reset_index()
    df1 = df1.pivot_table(index=['trade_dt'],columns=['industry'],values=['jingmairu'])['jingmairu']
    df1 = df1.iloc[-days:,:].sum()
    # 维修融资余额
    df2 = df2.copy()
    df2 = pd.merge(df2,index_chengfen,on=['scode']).drop(['industry','指数分类','代码'],axis=1).rename(columns={'指数简称':'industry'})
    # 融资余额
    df3 = df2.pivot_table(index=['trade_dt'],columns=['scode'],values=['guimo'])['guimo'].fillna(method='ffill')
    df3 = df3.reset_index().melt(id_vars=['trade_dt']).dropna().rename(columns={'value':'guimo'})
    info = df2[['scode','industry']].drop_duplicates()
    df2 = pd.merge(info,df3,on=['scode'])
    df2 = df2.groupby(['trade_dt','industry'])['guimo'].sum().reset_index()
    df2 = df2.pivot_table(index=['trade_dt'],columns=['industry'],values=['guimo'])['guimo'].diff()
    df2 = df2.iloc[-days:,:].sum()
    df = pd.concat([df1,df2],axis=1).rename(columns={0:'北上资金',1:'融资余额'})
    df = df[~df.index.str.contains('综合')]
    return df

In [15]:
hebing = get_hebing(waizi_specific,rzye,'北上资金',20)
x = hebing['北上资金'].tolist()
y = hebing['融资余额'].tolist()
color = hebing['北上资金'].rank().tolist()
text = hebing.index.tolist()
# 散点注释
annotations = []
for i in range(len(hebing)):
    tmp = {'x':hebing.iloc[i,0], 'y':hebing.iloc[i,1], 'xref':'x', 'yref':'y', 'text':hebing.index[i],'showarrow':True}
    annotations.append(tmp)

goplot(scatterGraph(x,y,text=text, #text是每个点的注释文字
                    yaxis='y2',name="2",
                    markerType = setMarker(color=color,colorscale='bluered'), #自动按照数值分配点的颜色
                    mode='markers+text',
                    textposition ='bottom center' # 注释显示的位置
                   ),
       setLayout(title='北上资金净流入与融资余额净流入20交易日关系',
                 xaxis=setAxis(title='北上资金总量'),
                 yaxis2=setAxis(title='融资余额')
                )
      )

In [16]:
hebing = get_hebing(waizi_specific,rzye,'配置',20)
x = hebing['北上资金'].tolist()
y = hebing['融资余额'].tolist()
color = hebing['北上资金'].rank().tolist()
text = hebing.index.tolist()
# 散点注释
annotations = []
for i in range(len(hebing)):
    tmp = {'x':hebing.iloc[i,0], 'y':hebing.iloc[i,1], 'xref':'x', 'yref':'y', 'text':hebing.index[i],'showarrow':True,'arrowhead':7}
    annotations.append(tmp)
    
goplot(scatterGraph(x,y,text=text, #text是每个点的注释文字
                    yaxis='y2',name="2",
                    markerType = setMarker(color=color,colorscale='bluered'), #自动按照数值分配点的颜色
                    mode='markers+text', #直接在图中显示注释
                    textposition ='bottom center' # 注释显示的位置
                   ), 
       setLayout(title='配置型资金净流入与融资余额净流入20交易日关系',
                 xaxis=setAxis(title='配置型北上资金'),
                 yaxis2=setAxis(title='融资余额')
                )
      )